In [2]:
  # Fix the strike price 
# Main thing 
# Create samples such that ;
# '''
# The four variables make a sample x is a data point x = (t, S, r, sigma) is data points. 
# No stable results -> then normalise the varaibles. 
#        t -> (0,2) only discrete number of days / 365. T is measure in years. 
#        S -> (50,150) stock prices. 
#        r -> [0%,1%] 
#        sigma (volatility) -> [1%,50%] -> standard deviation of the log of the returns. 
# We will use uniform distribution. 
# '''



In [3]:
# Work on this one feature at a time.  *

# Let say we want to make 500 samples to be safe. *

# Create 1 feature at a time column by column *

# then stack the columns together to get a matrix that is mxn and is check mate *

# create pricing function, use jacaboian and hessian to generate the data. *

# Do same procedure for the model. 

# investigate normalization after. Also how to print graphs with epochs and data. 

# Read the black scholes paper -> find ideas to relate to yours. 

In [42]:
pip install pytorch-lightning

     |████████████████████████████████| 915 kB 4.0 MB/s 
     |████████████████████████████████| 5.6 MB 14.8 MB/s 
     |████████████████████████████████| 118 kB 58.7 MB/s 
     |████████████████████████████████| 234 kB 67.5 MB/s 
     |████████████████████████████████| 636 kB 54.2 MB/s 
     |████████████████████████████████| 829 kB 60.9 MB/s 
     |████████████████████████████████| 1.3 MB 58.0 MB/s 
     |████████████████████████████████| 142 kB 50.2 MB/s 
     |████████████████████████████████| 294 kB 51.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=ef455b7d2d0636b8be6ed12e408084625f0efad79e1f99f332bd633fff826e28
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall

In [43]:
import numpy as np
import scipy.stats as si
import sympy as sy
from sympy.stats import Normal, cdf
from sympy import init_printing
init_printing()
import torch 
from torch import autograd
from torch import nn # import neural network 
from torch import optim 
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import pytorch_lightning as pl


In [5]:
# stock price - between [50,150]

a,b = 500 ,1 #dimension of the pytorch tensor to be generated
low , high = 50 , 150 #range of uniform distribution
torch.manual_seed(50) #Fx the values
S_t = torch.distributions.uniform.Uniform(low, high).sample([a,b]) # samples from low and high values. a,b are the dimensions

Stock = S_t.view(500,1)
print(Stock)

tensor([[111.8022],
        [ 56.8693],
        [ 88.9275],
        [ 54.0412],
        [ 90.1323],
        [ 64.4163],
        [ 96.0457],
        [ 98.7675],
        [109.2665],
        [146.3383],
        [ 62.3002],
        [ 90.4769],
        [ 99.8482],
        [149.8734],
        [110.4864],
        [102.2903],
        [119.7392],
        [ 75.0510],
        [ 86.2393],
        [ 96.2094],
        [121.4548],
        [100.5850],
        [ 55.1794],
        [ 74.9155],
        [ 73.9515],
        [ 92.3287],
        [ 50.2160],
        [118.4802],
        [124.9681],
        [ 74.8874],
        [ 84.9019],
        [ 69.5258],
        [ 77.9231],
        [ 75.2583],
        [ 87.9211],
        [126.8585],
        [119.0705],
        [125.2633],
        [ 61.8385],
        [136.9878],
        [133.9136],
        [ 55.3168],
        [ 67.0496],
        [ 79.1351],
        [106.8239],
        [ 53.7998],
        [ 77.1507],
        [119.8719],
        [ 59.4418],
        [117.1934],


In [6]:
# Time -[ between 0 and 2]

a,b = 500 ,1 #dimension of the pytorch tensor to be generated
low_t , high_t = 0, 730 #range of uniform distribution by number of days [0, 2]Years
torch.manual_seed(56) #Fx the values
t_t= torch.randint(low_t, high_t, size= (500,1) )  # https://pytorch.org/docs/stable/generated/torch.randint.html - generates uniformaly integers

t = t_t / 365 # to get a more reliable spread of results

# time for maturity 
#print(t)
Time = t.view(500,1)
#print(Time)

In [7]:

a,b = 500 ,1 #dimension of the pytorch tensor to be generated
low , high = 0 , 0.01 #range of uniform distribution [0%,1%]
torch.manual_seed(12) #Fx the values
r = torch.distributions.uniform.Uniform(low, high).sample([a,b]) # samples from low and high values. a,b are the dimensions

Rate = r.view(500,1)
print(Rate)

tensor([[4.6569e-03],
        [2.3277e-03],
        [4.5272e-03],
        [5.8711e-03],
        [4.0864e-03],
        [1.2717e-03],
        [6.3728e-03],
        [2.4206e-03],
        [7.3119e-03],
        [7.2244e-03],
        [1.9924e-03],
        [6.9483e-03],
        [5.8300e-03],
        [6.3183e-03],
        [5.5589e-03],
        [1.2625e-03],
        [9.7903e-03],
        [8.4427e-03],
        [1.2559e-03],
        [4.4562e-03],
        [6.6006e-03],
        [5.5381e-04],
        [1.5729e-03],
        [8.1367e-03],
        [7.2163e-03],
        [2.7173e-03],
        [3.0030e-03],
        [6.0994e-03],
        [5.7837e-03],
        [6.0834e-03],
        [4.3386e-03],
        [8.8131e-03],
        [3.2163e-03],
        [2.6044e-03],
        [2.5662e-03],
        [1.8722e-03],
        [6.4234e-03],
        [1.7856e-03],
        [1.4349e-03],
        [7.4904e-03],
        [7.2748e-03],
        [1.6413e-03],
        [3.2731e-03],
        [1.2394e-03],
        [6.1380e-03],
        [4

In [8]:
# volatility 

a,b = 500 ,1 #dimension of the pytorch tensor to be generated
low , high =  0.01, 0.5 #range of uniform distribution [1%,50%]
torch.manual_seed(30) #Fx the values
si = torch.distributions.uniform.Uniform(low, high).sample([a,b]) # samples from low and high values. a,b are the dimensions
# whenevr we want to fix a distribution we just use torch.manual_seed to fix the distribution.
Sigma = si.view(500,1)
print(Sigma) # 500 samples by 1 column

tensor([[0.4514],
        [0.3757],
        [0.2411],
        [0.4381],
        [0.3727],
        [0.3942],
        [0.4483],
        [0.3157],
        [0.2195],
        [0.4226],
        [0.3753],
        [0.3129],
        [0.4452],
        [0.2122],
        [0.0957],
        [0.1779],
        [0.4695],
        [0.0302],
        [0.0652],
        [0.3516],
        [0.4392],
        [0.1650],
        [0.1565],
        [0.3935],
        [0.3300],
        [0.2168],
        [0.1643],
        [0.2949],
        [0.0422],
        [0.1794],
        [0.4824],
        [0.1320],
        [0.0675],
        [0.3004],
        [0.2617],
        [0.2919],
        [0.1110],
        [0.1941],
        [0.0984],
        [0.2388],
        [0.0343],
        [0.2921],
        [0.0825],
        [0.1794],
        [0.1159],
        [0.1430],
        [0.4850],
        [0.4166],
        [0.0313],
        [0.2637],
        [0.1704],
        [0.2162],
        [0.1260],
        [0.1823],
        [0.1882],
        [0

In [9]:
# Stock, Time, Rate ,Sigma -> 4 features (variables)
# Strike
# https://discuss.pytorch.org/t/tensor-stack-or-concatenate/34331/6 - column and concatinate 

X = torch.column_stack([Stock, Time, Rate, Sigma])
print(X)
print(X.shape) # m x n matrix 


tensor([[1.1180e+02, 7.0959e-01, 4.6569e-03, 4.5136e-01],
        [5.6869e+01, 1.8685e+00, 2.3277e-03, 3.7572e-01],
        [8.8928e+01, 1.3342e+00, 4.5272e-03, 2.4109e-01],
        ...,
        [5.9128e+01, 1.7836e+00, 2.7279e-03, 4.5660e-01],
        [9.8155e+01, 7.0685e-01, 1.8035e-03, 2.5040e-01],
        [1.0653e+02, 1.7068e+00, 2.5701e-03, 3.3336e-01]])
torch.Size([500, 4])


In [10]:
X.requires_grad_(True) # graident for backprop activated with requires_grad true
X.retain_grad() # leaf tensor is a root of the tree 

print(X.shape) 

torch.Size([500, 4])


In [11]:
# non- divident paying asset. 
# P + S - C = K * e^-r(T-t)
# Set strike price = 100

Strike = 100 # fixed hyper paramters. 

# Our case this is a fixed paramter. 

In [12]:
def euro_vanilla_call(x):
  d1 = (torch.log(x[0] / Strike) + (x[2] + 0.5 * x[3] ** 2) * x[1]) / (x[3] * torch.sqrt(x[1]))
  d2 = (torch.log(x[0] / Strike) + (x[2] - 0.5 * x[3] ** 2) * x[1]) / (x[3] * torch.sqrt(x[1]))
    
  call = (x[0] * torch.distributions.normal.Normal(0,1).cdf(d1) - Strike * torch.exp(-x[2] * x[1]) * torch.distributions.normal.Normal(0,1).cdf(d2))

  return call
# https://pytorch.org/docs/stable/distributions.html - Normal( loc,scale)
#https://www.itl.nist.gov/div898/handbook/eda/section3/eda364.htm - loc and scale the same as mean and standard deviation.
# https://pytorch.org/docs/stable/generated/torch.log.html - returns natrual logarithem
# https://pytorch.org/docs/stable/generated/torch.sqrt.html - returns squart root


In [13]:
y = torch.zeros(size = (500 , 1)) # samples

for i in range(X.shape[0]):
  y[i] = euro_vanilla_call(X[i]) # runs through each row computation. 

print(y) # M samples dimension all in all. 

tensor([[ 2.2721e+01],
        [ 2.6445e+00],
        [ 6.0082e+00],
        [ 1.1345e+00],
        [ 1.3856e+01],
        [ 4.1641e+00],
        [ 1.2586e+01],
        [ 6.4811e+00],
        [ 1.7054e+01],
        [ 4.9546e+01],
        [ 5.2213e-01],
        [ 1.2441e+01],
        [ 2.4887e+01],
        [ 5.0325e+01],
        [ 1.1689e+01],
        [ 9.2247e+00],
        [ 3.5610e+01],
        [ 0.0000e+00],
        [ 1.4142e-05],
        [ 1.5222e+01],
        [ 3.1163e+01],
        [ 9.1199e+00],
        [ 0.0000e+00],
        [ 5.0606e+00],
        [ 6.1015e-01],
        [ 2.5259e+00],
        [ 3.0119e-04],
        [ 2.3009e+01],
        [ 2.5393e+01],
        [ 3.3681e-02],
        [ 1.7437e+01],
        [ 3.5364e-02],
        [ 4.3309e-03],
        [ 5.5197e+00],
        [ 8.1241e+00],
        [ 3.1608e+01],
        [ 1.9335e+01],
        [ 2.7152e+01],
        [ 7.7752e-04],
        [ 3.7977e+01],
        [ 3.5301e+01],
        [ 1.7973e-02],
        [ 1.8367e-05],
        [ 8

In [14]:
f_o = torch.zeros(size = (500,4))

# First order DY/DX

for i in range(X.shape[0]):
  f_o[i] = torch.autograd.functional.jacobian( euro_vanilla_call, X[i])

print(f_o.shape) # m samples by n features 
  
print(f_o) # call option. Theta is always negative as depreciation with the value of the option. And measures the variation.

torch.Size([500, 4])
tensor([[ 0.6887, 10.8391, 38.5158, 33.2855],
        [ 0.2022,  2.2232, 16.5465, 21.9077],
        [ 0.3973,  3.7116, 39.1187, 39.6127],
        ...,
        [ 0.2916,  3.5021, 21.8851, 27.0980],
        [ 0.5091,  5.9063, 30.0435, 32.9135],
        [ 0.6454,  5.1792, 80.7346, 51.7910]])


In [15]:
s_o = torch.zeros(size = (500,4,4))

#Second Order Risk D2Y/ DX2

for i in range(X.shape[0]):
  s_o[i] = torch.autograd.functional.hessian(euro_vanilla_call , X[i])

print(s_o.shape) # m samples by n features by n features .
  
print(s_o) # take logarithems if really small. Monotonic.

torch.Size([500, 4, 4])
tensor([[[ 8.3142e-03, -2.3563e-02,  6.5959e-01, -8.7696e-02],
         [-2.3563e-02, -7.1748e+00,  4.4718e+01,  2.4546e+01],
         [ 6.5959e-01,  4.4718e+01,  2.4998e+01, -3.0576e+01],
         [-8.7697e-02,  2.4546e+01, -3.0576e+01,  4.0653e+00]],

        [[ 9.6491e-03,  1.0288e-01,  1.0253e+00,  1.0106e+00],
         [ 1.0288e-01,  8.9427e-02,  1.5634e+01,  1.2530e+01],
         [ 1.0253e+00,  1.5634e+01,  7.8033e+01,  6.6451e+01],
         [ 1.0106e+00,  1.2530e+01,  6.6451e+01,  6.5497e+01]],

        [[ 1.5572e-02,  8.4154e-02,  1.8476e+00,  8.6206e-01],
         [ 8.4154e-02, -1.1207e+00,  3.4352e+01,  1.7096e+01],
         [ 1.8476e+00,  3.4352e+01,  1.6703e+02,  4.9431e+01],
         [ 8.6206e-01,  1.7096e+01,  4.9431e+01,  2.3063e+01]],

        ...,

        [[ 9.5176e-03,  1.1300e-01,  1.0037e+00,  8.7078e-01],
         [ 1.1300e-01, -3.3685e-01,  1.7941e+01,  1.2494e+01],
         [ 1.0037e+00,  1.7941e+01,  6.6816e+01,  4.3500e+01],
         [ 

In [17]:
# Data Collection

x = X.type(torch.FloatTensor)
y = y.type(torch.FloatTensor)
f_o = f_o.type(torch.FloatTensor)
s_o = s_o.type(torch.FloatTensor)

print(X.shape)
print(y.shape)
print(f_o.shape)
print(s_o.shape)

torch.Size([500, 4])
torch.Size([500, 1])
torch.Size([500, 4])
torch.Size([500, 4, 4])


In [53]:
# Pre-processing for pytorch. 

full_data = torch.utils.data.TensorDataset(X , y, f_o, s_o)
train, val = random_split(full_data, [400, 100])
train_loader = DataLoader(train, batch_size = 50, shuffle=False , num_workers =0) # hardcore
val_loader = DataLoader(val, batch_size = 50 , shuffle=False , num_workers =0)

#X_train = train
#X_val, y_val, f_o_val, s_o_val = val

In [57]:
# Paramters and hyperparamters 
input_size = 4 # x features
hidden_size_1 = 50
hidden_size_2 = 30
output_size = 1 # y output of training 

learning_rate = 0.001 # hyperparameter
num_epochs = 500 # hyperparameter    \ 250 ->  1 epoch
                                      # 250/32 = 6.2...    - > updating my weights every 6 training samples. 
#    SGD, ADAM, RMSPROP AND MORE .    Wi+1 = Wi - lr* gradient L / wrt to W   goes to minimum    -> update weights
#                                     Bi+1 = Bi - lr* gradient L / wrt to B  goes to minimum     -> Update biases 
alpha = 1 
beta = 0.5
delta = 0.25

In [58]:
class SecondOrderNetwork(pl.LightningModule):
  
  def __init__(self):
    super(SecondOrderNetwork,self).__init__()
    self.layer_1 = nn.Linear(input_size, hidden_size_1) # linear layer
    self.layer_2 = nn.Linear(hidden_size_1, hidden_size_2)
    self.layer_3 = nn.Linear(hidden_size_2, output_size)

    self.softplus = nn.Softplus() # absolute 
    self.loss = nn.MSELoss() # Possibility try different loss. Difference in log -> MSE(log([y]) - log([predicted_y)]) -> solves small

    self.automatic_optimization = False # Turn off automatic optimization. Turns off backprop automatically. 
# Automatic optimization = False does manual optimization not autoamtic - https://pytorch-lightning.readthedocs.io/en/latest/common/optimizers.html
  def forward(self, x):
    # layer 1
    x = self.layer_1(x)
    x = self.softplus(x)

    # layer 2
    x = self.layer_2(x)
    x = self.softplus(x)

    # layer 3 
    x = self.layer_3(x)

    return x


  def training_step(self, train_batch, batch_idx):
    opt = self.optimizers() # access optimizers thats defined below in optimization function
    opt.zero_grad() # clear gradients from the preavious training set (so only keeps paramters that updated)

    x, y, f_o, s_o = train_batch
    predicted_y = self.forward(x)
    # loop over batch
    predicted_f_o = torch.zeros(size=(x.shape[0], 4))
    for i in range(x.shape[0]): # x_shape is the batch size
      predicted_f_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True) # https://github.com/pytorch/pytorch/issues/46918 - Can show we need create_graph to be true as it backpropagets through the graph

    predicted_s_o = torch.zeros(size=(x.shape[0], 4, 4))
    for i in range(x.shape[0]): # x_shape is the batch size
      predicted_s_o[i] = torch.autograd.functional.hessian(self.forward, x[i], create_graph=True, strict=True)
    # create_graph = True makes derivatives differentiable through the chain using back prop. 
    # strict = True makes sure all inputs are connected to the outputs ortherwise an error will occur. If false it will be all zeros.
    loss = alpha * self.loss(y, predicted_y) + beta * self.loss( f_o, predicted_f_o) + delta * self.loss(s_o,  predicted_s_o)

    loss.backward(retain_graph=True) # Saves gradeints and not deleted them after first backprop . 

    opt.step() # Every batch update your paramters using optimizer step. 

    self.log('train_loss', loss, on_step = False, on_epoch = True)
    print(loss)
    return loss
    
  def validation_step(self, val_batch, batch_idx):
    x, y, f_o, s_o = val_batch
    predicted_y = self.forward(x)
    # loop over batch
    predicted_f_o = torch.zeros(size=(x.shape[0], 4))
    for i in range(x.shape[0]): # validatio batch size
      predicted_f_o[i] = torch.autograd.functional.jacobian(self.forward, x[i], create_graph=True, strict=True)

    predicted_s_o = torch.zeros(size=(x.shape[0], 4, 4))
    for i in range(x.shape[0]): # validation batch size
      predicted_s_o[i] = torch.autograd.functional.hessian(self.forward, x[i], create_graph=True, strict=True)

    loss = alpha * self.loss(y, predicted_y) + beta * self.loss( f_o, predicted_f_o) + delta * self.loss(s_o,  predicted_s_o)
    self.log('val_loss', loss) # no optimization required for
   # print( loss) 
    return loss

  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-2)


In [62]:
trainer = pl.Trainer(max_epochs= 300)
# https://www.machinecurve.com/index.php/question/how-to-set-number-of-epochs-in-pytorch-lightning/
model = SecondOrderNetwork() # Trained. 

trainer.fit(model, train_loader, val_loader) 

# take lograithems 

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name     | Type     | Params
--------------------------------------
0 | layer_1  | Linear   | 250   
1 | layer_2  | Linear   | 1.5 K 
2 | layer_3  | Linear   | 31    
3 | softplus | Softplus | 0     
4 | loss     | MSELoss  | 0     
--------------------------------------
1.8 K     Trainable params
0         Non-trainable params
1.8 K     Total params
0.007     Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:323: UserWarning: The number of training samples (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


tensor(1936.6995, grad_fn=<AddBackward0>)
tensor(1829.4332, grad_fn=<AddBackward0>)
tensor(1895.9531, grad_fn=<AddBackward0>)


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
